In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader -> datasets 라이브러리랑 충돌

c:\Users\ehddl\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer-wO1G7-CE-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/")

In [12]:
data = pd.read_csv("tests/data/final_fine-tuning_multi-columns_data.csv", index_col=0)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
# train, test = train_test_split(data, test_size= 0.2, stratify=data['label_id'], random_state=42)

In [ ]:
# # huggingface의 datasets.Dataset을 사용하는 방식

# dataset = DatasetDict({
#     'train' : Dataset.from_pandas(train),
#     'test' : Dataset.from_pandas(test)
# })

In [18]:
model_name = "kykim/bert-kor-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
def tokenize_three_columns(examples):
    """
    세 개의 텍스트 컬럼을 [SEP] 토큰으로 연결하여 토큰화하는 함수.
    
    Args:
        examples (dict): Hugging Face Dataset의 batch 데이터를 담고 있는 딕셔너리.
                         `acnt_sub_nm_cleaned`, `intro_txt_cleaned`, `text` 키를 포함해야 합니다.
    
    Returns:
        dict: 토큰화된 'input_ids', 'attention_mask'를 담고 있는 딕셔너리.
    """
    
    # 텍스트 컬럼들을 [SEP] 토큰으로 연결하여 하나의 시퀀스로 만듭니다.
    # f-string을 사용하여 간결하게 결합합니다.
    # BERT는 "[CLS] text1 [SEP] text2 [SEP] text3 [SEP]" 형태의 입력을 처리합니다.
    combined_texts = [
        f"{acnt} {tokenizer.sep_token} {intro} {tokenizer.sep_token} {txt}"
        for acnt, intro, txt in zip(
            examples["acnt_sub_nm_cleaned"],
            examples["intro_txt_cleaned"],
            examples["text"]
        )
    ]
    
    return tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512  # BERT의 최대 입력 길이
    )